In [18]:
!pip install tensorflow keras
!pip install pillow
!pip install tensorflow-model-optimization tf-keras
!pip install scikit-learn

In [19]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam



In [20]:
# Define paths
dataset_dir = 'dataset'
train_dir = dataset_dir 
val_dir = dataset_dir  # You can split data into train/val if needed

# Image parameters
img_height, img_width = 96, 96  # Adjust as needed
batch_size = 32


# Data augmentation
train_datagen = ImageDataGenerator()
val_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='categorical'  # Use 'binary' for 2 classes
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='categorical'  # Use 'binary' for 2 classes
)

Found 3000 images belonging to 3 classes.
Found 3000 images belonging to 3 classes.


In [21]:
from tensorflow.keras import saving

@saving.register_keras_serializable()
def F1_score(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

model = tf.keras.models.load_model('outputs/out_v4.keras')



In [22]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import numpy as np

# Define a representative dataset generator
def representative_dataset():
    dataset_list = tf.data.Dataset.list_files(dataset_dir + '/*/*')
    for i in range(100):
        image_path = next(iter(dataset_list)).numpy().decode("utf-8")
        img = load_img(image_path, target_size=(img_height, img_width))
        img = img_to_array(img)
        img = np.expand_dims(img, axis=0)
        yield [img.astype(np.float32)]
        
# Create the TFLite converter with INT8 quantization and representative dataset
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]  
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
converter.representative_dataset = representative_dataset
tflite_model = converter.convert()   

# Save the TFLite model
with open('outputs/mobilenetv4_model_int8.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpopta02xo/assets


INFO:tensorflow:Assets written to: /tmp/tmpopta02xo/assets


Saved artifact at '/tmp/tmpopta02xo'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 96, 96, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  140366445920720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140366445921872: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140366445922256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140366445921680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140366445921104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140366445922064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140366445924560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140366445924752: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140366445924368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140366445923792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140366445925136

/home/nikolai/anaconda3/envs/train_p11/lib/python3.11/site-packages/tensorflow/lite/python/convert.py:997: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
W0000 00:00:1733267561.777776     408 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1733267561.778019     408 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2024-12-04 00:12:41.779074: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpopta02xo
2024-12-04 00:12:41.785747: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-12-04 00:12:41.785761: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpopta02xo
2024-12-04 00:12:41.870878: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-12-04 00:12:42.148106: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at

In [32]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import numpy as np

# Testing block
def test_image(image_path):
    """
    Loads an image, preprocesses it, and predicts its class using the trained model.

    Args:
        image_path (str): Path to the image file.
    """
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)  # Add batch dimension

    prediction = model.predict(img)
    class_index = np.argmax(prediction)
    class_labels = list(train_generator.class_indices.keys())  # Get class labels
    confidence = np.max(prediction)  # Get the highest probability

    print(prediction);
    print(class_labels);
    print("Predicted class: %s (confidence: %.2f)" % (class_labels[class_index], confidence))


# Example usage
test_image('cat-test.jpg') 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
[[0.761813   0.14891128 0.08927564]]
['cat', 'elephant', 'horse']
Predicted class: cat (confidence: 0.76)


In [33]:
# Verify the TFLite model
interpreter = tf.lite.Interpreter(model_path="outputs/mobilenetv4_model_int8.tflite")
interpreter.allocate_tensors()

def test_image_tflite(image_path):
    categories = np.array(["cat", "elephant", "horse"]);
    
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    input_scale, input_zero_point = input_details[0]['quantization']
    
    # Load the custom image
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    
    image_array_uint8 = (img / input_scale + input_zero_point).astype("uint8")
    image_array_uint8 = np.expand_dims(image_array_uint8, axis=0)  # Add batch dimension
    
    # Set the input tensor
    interpreter.set_tensor(input_details[0]['index'], image_array_uint8)
    
    # Invoke the model
    interpreter.invoke()
    
    # Get the predictions
    predictions = interpreter.get_tensor(output_details[0]['index'])
    print(predictions)
    print("Predictions: %s with confidence %.2f" % (categories[predictions.argmax()], (predictions[0][predictions.argmax()])/255*100))

test_image_tflite("elephant-test.jpg")

[[ 27 206  23]]
Predictions: elephant with confidence 80.78


In [25]:
from tensorflow.lite.python.util import convert_bytes_to_c_source

file_name = 'model'
source_text, header_text = convert_bytes_to_c_source(tflite_model,  file_name)

with  open('outputs/' + file_name + '.h',  'w')  as  file:
    file.write(header_text)

with  open('outputs/' + file_name + '.cc',  'w')  as  file:
    file.write(source_text)
